In [1]:
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install torch torchvision torchaudio av huggingface_hub

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-10cs5k9o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-10cs5k9o
  Resolved https://github.com/huggingface/transformers.git to commit dc06f2dd2e3b7a4ce3f49392ac405307a9634355
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 20.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=11083804 sha256=38c210845435bf0bb4044139ad7f835bd94ccc84fb8e6c2e6f11ffc725155e96
  Stored in directory: /tmp/pip-ephem-wheel-cache-toc50r7i/wheels/12/51/df/b62c8ce0479c5de6f7bef121169b3e946949a57481169d3155
Successfu

In [2]:
# Imports
import torch
from transformers import PeAudioVideoModel, PeAudioVideoProcessor
from torch.nn.functional import cosine_similarity

In [3]:
# Load model
model = PeAudioVideoModel.from_pretrained("facebook/pe-av-small")
processor = PeAudioVideoProcessor.from_pretrained("facebook/pe-av-small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.39G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/859 [00:00<?, ?it/s]

processor_config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

PeAudioVideoModel(
  (text_model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 1024, padding_idx=50283)
      (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=1024, out_features=3072, bias=False)
          (Wo): Linear(in_features=1024, out_features=1024, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=1024, out_features=5248, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=2624, out_features=1024, bias=False)
        )
      )
      (1-21): 21 x ModernBertEncoderLayer(
        (att

In [24]:
videos = ["video1.mp4", "video2.mp4"]
audios = ["audio1.mp3", "audio2.mp3"]




In [27]:
pairs = [(v, a) for v in videos for a in audios]
embeddings = {}
for vfile, afile in pairs:
    inputs = processor(videos=[vfile], audio=[afile], return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**inputs)
    embeddings[(vfile, afile)] = out.audio_video_embeds.cpu()

In [28]:
# Compare embeddings (cosine similarity)
print("Similarity scores:")
for (v1, a1), emb1 in embeddings.items():
    for (v2, a2), emb2 in embeddings.items():
        score = cosine_similarity(emb1, emb2).item()
        print(f"{v1}+{a1} <-> {v2}+{a2} : {score:.4f}")

Similarity scores:
video1.mp4+audio1.mp3 <-> video1.mp4+audio1.mp3 : 1.0000
video1.mp4+audio1.mp3 <-> video1.mp4+audio2.mp3 : 0.8721
video1.mp4+audio1.mp3 <-> video2.mp4+audio1.mp3 : 0.8666
video1.mp4+audio1.mp3 <-> video2.mp4+audio2.mp3 : 0.7425
video1.mp4+audio2.mp3 <-> video1.mp4+audio1.mp3 : 0.8721
video1.mp4+audio2.mp3 <-> video1.mp4+audio2.mp3 : 1.0000
video1.mp4+audio2.mp3 <-> video2.mp4+audio1.mp3 : 0.7447
video1.mp4+audio2.mp3 <-> video2.mp4+audio2.mp3 : 0.8583
video2.mp4+audio1.mp3 <-> video1.mp4+audio1.mp3 : 0.8666
video2.mp4+audio1.mp3 <-> video1.mp4+audio2.mp3 : 0.7447
video2.mp4+audio1.mp3 <-> video2.mp4+audio1.mp3 : 1.0000
video2.mp4+audio1.mp3 <-> video2.mp4+audio2.mp3 : 0.8824
video2.mp4+audio2.mp3 <-> video1.mp4+audio1.mp3 : 0.7425
video2.mp4+audio2.mp3 <-> video1.mp4+audio2.mp3 : 0.8583
video2.mp4+audio2.mp3 <-> video2.mp4+audio1.mp3 : 0.8824
video2.mp4+audio2.mp3 <-> video2.mp4+audio2.mp3 : 1.0000
